# Decimate data and save as miniSEED
This decimates the 200Hz DAS data from Paradise (Mt Rainier) and saves it as miniSEED. The decimation is not done rigorously with previous filtering, but it's good enough to pick the signal from cars on the road.

In [46]:
# load modules
import glob
import numpy as np
import h5py
import os
import datetime
import obspy

## Read and decimate raw hdf5 data
The data is not provided in the repository, where this notebook is located. To reproduce the analysis, recorded files from 2023-11-24 17:20:00 UTC until 2023-11-24 17:59:00 UTC are needed.

In [47]:
# Get file paths
files = glob.glob('../data/recordings/raw_data/*.h5')
files.sort()

In [48]:
# Set decimation factor
dec_factor = 20  # 200Hz -> 10Hz

In [49]:
# Read data and decimate
data_list = []
time_list = []

for i,file in enumerate(files):
    print("File {} of {}".format(i+1, len(files)), end="\r")

    # Read the data (really simple and stupid)
    f = h5py.File(file,'r')
    time_read = np.array(f['Acquisition/Raw[0]/RawDataTime'], dtype='int64')
    data_read = np.array(f['Acquisition/Raw[0]/RawData'], dtype='float32')   
    attrs_read = dict(f['Acquisition'].attrs)
    f.close()
    
    # Decimate
    data_list.append(data_read[::dec_factor,:])
    time_list.append(time_read[::dec_factor])

In [50]:
# Combine decimated data into one data array and one time array
data_all = np.concatenate(data_list, axis=0)
time_all = np.concatenate(time_list, axis=0)

In [51]:
# Adjust sampling rates in attributes
attrs = attrs_read.copy()
attrs['PulseRate'] = attrs_read['PulseRate']/dec_factor
attrs['MaximumFrequency'] = attrs_read['MaximumFrequency']/dec_factor

## Write DAS data to miniSEED
The data is written into an obspy.Stream() and saved as a single miniSEED file. Note that the channel name of a single trace can only have three characters, but we have thousands of DAS channels. Therefore, the channel number is written into the stats.station attribute.

In [52]:
# Write data into obspy Stream and save as miniSEED
st = obspy.Stream()
for i in range(data_all.shape[1]):
    # st = obspy.Stream()
    tr = obspy.Trace()
    # set the stats
    stats = obspy.core.trace.Stats()
    stats.network = 'UW'
    stats.station = '{:04d}'.format(i)
    stats.location = ''
    stats.channel = ''
    stats.sampling_rate = attrs['PulseRate']
    stats.npts = time_all.shape[0]
    stats.starttime = obspy.UTCDateTime(datetime.datetime.utcfromtimestamp(time_all[0]/1e6))
    stats.gauge_length = attrs['GaugeLength']
    stats.spatial_sampling_interval = attrs['SpatialSamplingInterval']
    # add data
    tr.data = data_all[:,i]
    tr.stats = stats
    # append trace to stream
    st.append(tr)
st.sort()
    
filename = 'rainier_10Hz.mseed'
savepath = os.path.join('../data/recordings/',filename)
st.write(savepath, format="MSEED")

## Now it's picking time
Use the Snuffler tool from the pyrocko python package to pick the signals from Dominik and Manuela's car in in the DAS data. Take a look at the gpx track as a reference to find which car is the correct one.